In [2]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.2 MB/s eta 0:00:00:00:0100:01


In [ ]:
# =====================================================
# 1️⃣ SETUP
# =====================================================
import os, zipfile, shutil, yaml
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont

# =====================================================
# 0️⃣ PERSISTENT FOLDER SETUP
# =====================================================
PERSISTENT_ROOT = "/kaggle/working/persistent_outputs"
os.makedirs(PERSISTENT_ROOT, exist_ok=True)

# Persistent subfolders
PERSISTENT_RUNS = os.path.join(PERSISTENT_ROOT, "helmet_yolov8_runs")
PERSISTENT_BUNDLE = os.path.join(PERSISTENT_ROOT, "light_bundle")
os.makedirs(PERSISTENT_RUNS, exist_ok=True)
os.makedirs(PERSISTENT_BUNDLE, exist_ok=True)



# =====================================================
# 1a UNZIP & FLATTEN DATASET
# =====================================================
import os, shutil, yaml

uploaded_zip = "/kaggle/input/helmet-dataset-zip"  # update with actual zip path
dataset_root = "/kaggle/working/dataset"
os.makedirs(dataset_root, exist_ok=True)

# 1b Detect nested Google Drive structure and flatten
nested_root = None
for root, dirs, files in os.walk(uploaded_zip):
    if 'images' in dirs and 'labels' in dirs and 'data.yaml' in files:
        nested_root = root
        break

if nested_root is None:
    raise RuntimeError("Could not find dataset structure with images/, labels/, data.yaml")

# 1c Copy images and labels to flat structure (reset clean each run)
flat_dataset_dir = "/kaggle/working/flat_dataset"
os.makedirs(flat_dataset_dir, exist_ok=True)

for subdir in ["images", "labels"]:
    dest = os.path.join(flat_dataset_dir, subdir)
    if os.path.exists(dest):
        shutil.rmtree(dest)  # remove old copy
    shutil.copytree(os.path.join(nested_root, subdir), dest)

# Always refresh data.yaml
yaml_src = os.path.join(nested_root, "data.yaml")
yaml_dest = os.path.join(flat_dataset_dir, "data.yaml")
if os.path.exists(yaml_dest):
    os.remove(yaml_dest)
shutil.copy(yaml_src, flat_dataset_dir)

# 1d Update data.yaml paths
data_yaml_path = os.path.join(flat_dataset_dir, "data.yaml")
with open(data_yaml_path) as f:
    data_yaml = yaml.safe_load(f)

# 1e Update train/val/test paths to flattened structure
data_yaml['train'] = os.path.join(flat_dataset_dir, "images/train")
data_yaml['val'] = os.path.join(flat_dataset_dir, "images/val")
data_yaml['test'] = os.path.join(flat_dataset_dir, "images/test")

with open(data_yaml_path, 'w') as f:
    yaml.safe_dump(data_yaml, f)

DATA_YAML_PATH = data_yaml_path
print(f"✅ Dataset prepared at: {flat_dataset_dir}")


# =====================================================
# 2️ TRAINING WITH AUTO-DETECT RESUME
# =====================================================
OUTPUT_DIR = "/kaggle/working/helmet_yolov8_runs"
LIGHT_BUNDLE = "/kaggle/working/light_bundle"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LIGHT_BUNDLE, exist_ok=True)
# Redirect workflow outputs
OUTPUT_DIR = PERSISTENT_RUNS
LIGHT_BUNDLE = PERSISTENT_BUNDLE
# Auto-detect checkpoint: prefer last.pt, fallback to best.pt
last_ckpt_path = os.path.join(LIGHT_BUNDLE, "last.pt")
best_ckpt_path = os.path.join(LIGHT_BUNDLE, "best.pt")

if os.path.exists(last_ckpt_path):
    ckpt_to_resume = last_ckpt_path
    print(f"✅ Resuming training from latest checkpoint: {ckpt_to_resume}")
elif os.path.exists(best_ckpt_path):
    ckpt_to_resume = best_ckpt_path
    print(f"✅ Resuming training from best checkpoint: {ckpt_to_resume}")
else:
    ckpt_to_resume = None
    print(" No checkpoint found, starting training from scratch")

model = YOLO(ckpt_to_resume if ckpt_to_resume else "yolov8s.pt")

results = model.train(
    data=DATA_YAML_PATH,
    epochs=100,
    imgsz=640,
    batch=24,
    optimizer="AdamW",
    lr0=1e-3,
    patience=20,          # early stopping patience
    amp=True,
    close_mosaic=30,     # disables mosaic augmentation for last 30 epochs
    project=OUTPUT_DIR,
    name="exp",
    exist_ok=True,
    save_period=5
)



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Dataset prepared at: /kaggle/working/flat_dataset
 No checkpoint found, starting training from scratch
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=30, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/flat_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0,

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

train: Scanning /kaggle/working/flat_dataset/labels/train... 4916 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4916/4916 [00:03<00:00, 1480.54it/s]


train: New cache created: /kaggle/working/flat_dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 549.6±280.3 MB/s, size: 61.2 KB)


val: Scanning /kaggle/working/flat_dataset/labels/val... 1413 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1413/1413 [00:01<00:00, 1344.72it/s]

...

                   all       1413       5252      0.957      0.925      0.972      0.683

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     92/100      6.93G     0.8518     0.3486     0.9835         66        640: 100%|██████████| 205/205 [01:22<00:00,  2.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:12<00:00,  2.49it/s]

                   all       1413       5252      0.958      0.923      0.971      0.683



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     93/100      6.93G     0.8471     0.3482     0.9793         64        640: 100%|██████████| 205/205 [01:22<00:00,  2.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  43%|████▎     | 13/30 [00:05<00:06,  2.48it/s]

In [21]:
# Copy checkpoints into light bundle
RUN_DIR = "/kaggle/working/persistent_outputs/helmet_yolov8_runs/exp/weights"
shutil.copy(os.path.join(RUN_DIR, "best.pt"), LIGHT_BUNDLE)
shutil.copy(os.path.join(RUN_DIR, "last.pt"), LIGHT_BUNDLE)


# =====================================================
# 3️ SAVE TRAINING RESULTS + PLOTS
# =====================================================
csv_path = os.path.join("/kaggle/working/persistent_outputs/helmet_yolov8_runs/exp", "results.csv")
df = pd.read_csv(csv_path)
df.to_csv(os.path.join(LIGHT_BUNDLE, "results.csv"), index=False)

# Loss curves
plt.figure()
plt.plot(df["epoch"], df["train/box_loss"], label="Box Loss")
plt.plot(df["epoch"], df["train/cls_loss"], label="Cls Loss")
plt.plot(df["epoch"], df["train/dfl_loss"], label="DFL Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Training Losses")
plt.legend(); plt.grid()
plt.savefig(os.path.join(LIGHT_BUNDLE, "loss_curves.png")); plt.close()

# Metrics curves
plt.figure()
plt.plot(df["epoch"], df["metrics/precision(B)"], label="Precision")
plt.plot(df["epoch"], df["metrics/recall(B)"], label="Recall")
plt.plot(df["epoch"], df["metrics/mAP50(B)"], label="mAP@50")
plt.plot(df["epoch"], df["metrics/mAP50-95(B)"], label="mAP@50-95")
plt.xlabel("Epoch"); plt.ylabel("Metric"); plt.title("Validation Metrics")
plt.legend(); plt.grid()
plt.savefig(os.path.join(LIGHT_BUNDLE, "metrics_curves.png")); plt.close()


# =====================================================
# 0 TESTING ON TEST SPLIT
# =====================================================
pred = model.predict(
    source = "/kaggle/working/flat_dataset/images/test",
    save=True,
    project="/kaggle/working/persistent_outputs/helmet_yolov8_runs",
    name="exp/predict_test",
    imgsz=640,
    conf=0.001
)

# =====================================================
# 6. SAVE RANDOM 10 PREDICTION OVERLAYS (using YOLO .plot())
# =====================================================
import cv2
import random
import os
import shutil
import zipfile

# Directory for overlays
output_dir = os.path.join(LIGHT_BUNDLE, "pred_overlays")

# Remove old overlays if they exist
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# Pick 10 random predictions
sampled_preds = random.sample(pred, min(10, len(pred)))

# Save overlays
for pred_obj in sampled_preds:
    img_overlay = pred_obj.plot()  # YOLO's built-in overlay
    out_name = os.path.splitext(os.path.basename(pred_obj.path))[0] + "_overlay.jpg"
    cv2.imwrite(os.path.join(output_dir, out_name), img_overlay)

print(f"✅ Overlays saved in: {output_dir}")


# =====================================================
# 7. UPDATE LIGHT BUNDLE ZIP
# =====================================================
zip_base = "/kaggle/working/persistent_outputs/light_bundle"
zip_path = f"{zip_base}.zip"

# Remove old zip if it exists
if os.path.exists(zip_path):
    os.remove(zip_path)

# Create fresh zip (this includes training results + updated overlays)
shutil.make_archive(zip_base, "zip", LIGHT_BUNDLE)

print(f"✅ Light bundle updated: {zip_path}")



image 1/706 /kaggle/working/flat_dataset/images/test/005298_jpg.rf.ef704ea8092352ea3e5c2c41c0f701dd.jpg: 640x640 21 with_helmets, 16.3ms
image 2/706 /kaggle/working/flat_dataset/images/test/005299_jpg.rf.8b93906788e4ec2be34e04b1bc712f39.jpg: 640x640 4 with_helmets, 16.2ms
image 3/706 /kaggle/working/flat_dataset/images/test/005300_jpg.rf.26ba2d0b2ce4c5a268c196b8ee53e9e1.jpg: 640x640 4 with_helmets, 16.2ms
image 4/706 /kaggle/working/flat_dataset/images/test/005301_jpg.rf.18cd1a6aa3cbd6cad27655caeefff795.jpg: 640x640 15 with_helmets, 16.2ms
image 5/706 /kaggle/working/flat_dataset/images/test/005302_jpg.rf.2724b46fc737a786d0dabb343e04e5d1.jpg: 640x640 14 with_helmets, 16.2ms
image 6/706 /kaggle/working/flat_dataset/images/test/005303_jpg.rf.f878712db020a544be5ab439981b09a2.jpg: 640x640 11 with_helmets, 16.2ms
image 7/706 /kaggle/working/flat_dataset/images/test/005304_jpg.rf.0d10d9b882ef7d1077be5b7b5588c5fa.jpg: 640x640 13 with_helmets, 16.2ms
image 8/706 /kaggle/working/flat_dataset/i

In [18]:
# =====================================================
# 6 PACKAGE LIGHT BUNDLE
# =====================================================
zip_path = "/kaggle/working/persistent_outputs/helmet_yolov8_runs.zip"
with zipfile.ZipFile(zip_path, "w") as z:
    for root, _, files in os.walk("/kaggle/working/persistent_outputs/helmet_yolov8_runs"):
        for file in files:
            z.write(os.path.join(root, file),
                    os.path.relpath(os.path.join(root, file), "/kaggle/working/persistent_outputs/helmet_yolov8_runs"))
print(f"✅ Light bundle created: {zip_path}")

✅ Light bundle created: /kaggle/working/persistent_outputs/helmet_yolov8_runs.zip
